# Test sentiment model

In [1]:
from sentence_transformers import util

In [2]:
from utils import sentiment

In [ ]:
sentim = sentiment.Sentiment()

You try to use a model that was created with version 2.6.1, however, your version is 2.5.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

C:\Users\pavel\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pavel\.cache\huggingface\hub\models--pavlentiy--reviews-sentiment-multilingual-e5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [30]:
import numpy as np
from sentence_transformers import SentenceTransformer, util

sentences = ["Замечательный препарат, все пользуемся", "Последнее время данный препарат вызывает у меня сыпь"]
classes = ['негатив', 'нейтрально', 'позитив']


model = SentenceTransformer('pavlentiy/reviews-sentiment-multilingual-e5-base')
embeddings = model.encode(sentences)
embeddings_classes = model.encode(classes)

# Compute cosine-similarities
cosine_scores = np.array(util.cos_sim(embeddings, embeddings_classes))

a = lambda t: {0:'негатив', 1:'нейтральная', 2:'позитив'}[t]
argmax = cosine_scores.argmax(axis=1)
result_classes = list(map(a, argmax))
print(result_classes)

['позитив', 'негатив']


KeyError: tensor(2)

In [19]:
a(argmax)

TypeError: unhashable type: 'numpy.ndarray'

# Test BOT

In [6]:
import telebot
import pandas as pd
import numpy as np
from utils import sentiment
from utils import preprocessing

import io
import telebot
from telebot import types

import warnings
import logging

warnings.filterwarnings("ignore")

logging.basicConfig(level=logging.INFO)

# bot = telebot.TeleBot('7004638154:AAHSXlAn6hhu999TmK9Nkzc4AIoVJHXW9GA') # prom stand
bot = telebot.TeleBot('7139634075:AAG_TtOalJPAUqSnfXLEQknNe0QkiPQpTfQ') # ift stand

preproc = preprocessing.Preprocessing
# sentim = sentiment.Sentiment()
sentim = sentiment.Sentiment_TEST()

@bot.message_handler(commands=['start', 'button'])
def welcome(message):
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    bot.send_message(message.chat.id, "Определение тональности текстов\nДобавьте файл в формате .xlsx\nСтолбец с текстом для обработки должен называть 'Текст сообщения'")


@bot.message_handler(content_types=['document'])
def handle_docs_photo(message):
    try:
        chat_id = message.chat.id

        file_info = bot.get_file(message.document.file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        df = pd.read_excel(downloaded_file)
        
        minutes = round(df.shape[0] * 1.2,) // 60
        seconds = round(df.shape[0] * 1.2,) % 60
        bot.reply_to(message, f"Взял в обработку, нужно немного времени\nПримерное время обработки {minutes}:{seconds}")
        
        redused = df[~df['Текст сообщения'].isna()]
        redused['Текст сообщения'] = redused['Текст сообщения'].apply(lambda x: preproc.soft_preprocessing(x))
        sentences = redused['Текст сообщения'].tolist()
        sentiment_result, sureness = sentim.get_sentiment(sentences)
        redused['model sentiment'] = sentiment_result
        redused['model sureness'] = sureness
        
        df = df.join(redused[['model sentiment', 'model sureness']])
            
        buf = io.BytesIO()
        
        df.to_excel(buf, encoding='utf-8', index = False, header = True, )
        
        filename_old = message.document.file_name
        filename_new = filename_old.split('.')[0] + '__processed__.' + filename_old.split('.')[1]
        
        bot.send_document(chat_id, buf.getvalue(), visible_file_name=filename_new)
#         bot.send_message(message.chat.id, text="Файл обработан. Переименуйте его с расширением .xlsx")
        buf.close()
        
    except Exception as e:
        bot.reply_to(message, e)    
    
logging.info('Server started')
bot.polling(none_stop=True, interval=0)

INFO:root:Server started


'Цитобронхо_март__processed__.xlsx'